<table class="tfo-notebook-buttons" align="center" style="width: 90%; border-collapse: collapse; margin: 10px auto;">
  <tr>
    <td style="width: 33%; text-align: center; padding: 10px;">
      <a target="_blank" href="https://colab.research.google.com/github/subornaa/Data-Analytics-Tutorials/blob/main/Data_Wrangling_Tutorials.ipynb">
        <img src="https://tensorflow.google.cn/images/colab_logo_32px.png" alt="Colab Logo" style="vertical-align: middle; margin-right: 5px;">
        <span style="font-size: 14px; font-weight: bold;">Run in Google Colab</span>
      </a>
    </td>
    <td style="width: 33%; text-align: center; padding: 10px;">
      <a target="_blank" href="https://github.com/subornaa/Data-Analytics-Tutorials/blob/main/Data_Wrangling_Tutorials.ipynb">
        <img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" alt="GitHub Logo" style="vertical-align: middle; margin-right: 5px;">
        <span style="font-size: 14px; font-weight: bold;">View on GitHub</span>
      </a>
    </td>
    <td style="width: 33%; text-align: center; padding: 10px;">
      <a href="https://drive.google.com/file/d/1UDKAdXW0h6JSf7k31PZ-srrQ3487l9e2">
        <img src="https://tensorflow.google.cn/images/download_logo_32px.png" alt="Download Logo" style="vertical-align: middle; margin-right: 5px;">
        <span style="font-size: 14px; font-weight: bold;">Download Data</span>
      </a>
    </td>
  </tr>
</table>